## This code will extract 6 channels from psg edf file(sub-1_task-Sleep_acq-psg_eeg.edf)  and store it as new (sub-1_task-Sleep_acq-psg_eeg_6-channels.edf)

ok, let's try this for 1 file first 

In [16]:
sub_no = 29

In [2]:
#pip install mne
#pip install pyedflib

In [1]:
import mne
import pyedflib
import numpy as np


In [4]:

file_path = f'Dataset_clean_for_jupyter/sub-{sub_no}/eeg/sub-{sub_no}_task-Sleep_acq-psg_eeg.edf'
raw = mne.io.read_raw_edf(file_path, preload=True)
print(raw)
print(raw.info)

Extracting EDF parameters from C:\Users\naikh\SleepResearchCode_experient\Dataset_clean_for_jupyter\sub-29\eeg\sub-29_task-Sleep_acq-psg_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8510463  =      0.000 ... 33243.996 secs...


C:\Users\naikh\AppData\Local\Temp\ipykernel_6448\2810092197.py:2: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(file_path, preload=True)


<RawEDF | sub-29_task-Sleep_acq-psg_eeg.edf, 14 x 8510464 (33244.0 s), ~909.0 MiB, data loaded>
<Info | 8 non-empty values
 bads: []
 ch_names: PSG_F3, PSG_F4, PSG_C3, PSG_C4, PSG_O1, PSG_O2, PSG_EOG, ...
 chs: 14 EEG
 custom_ref_applied: False
 highpass: 0.1 Hz
 lowpass: 128.0 Hz
 meas_date: 2023-02-09 22:02:56 UTC
 nchan: 14
 projs: []
 sfreq: 256.0 Hz
 subject_info: <subject_info | his_id: sub-29>
>


In [5]:
raw.ch_names

['PSG_F3',
 'PSG_F4',
 'PSG_C3',
 'PSG_C4',
 'PSG_O1',
 'PSG_O2',
 'PSG_EOG',
 'PSG_EMG',
 'PSG_THER',
 'PSG_THOR',
 'PSG_ABD',
 'PSG_PULSE',
 'PSG_BEAT',
 'PSG_SPO2']

In [6]:
raw2 = raw.copy().pick(range(6))

In [7]:
raw2.ch_names

['PSG_F3', 'PSG_F4', 'PSG_C3', 'PSG_C4', 'PSG_O1', 'PSG_O2']

In [8]:
# Define the new file path
new_file_path =f'Dataset_clean_for_jupyter/sub-{sub_no}/eeg/sub-{sub_no}_task-Sleep_acq-psg_eeg_6-channels.edf'

# Save the modified data as a new EDF file
##raw2.save(new_file_path,fmt='edf', overwrite=True)

In [13]:


# Create a new EDF file
n_channels = len(raw2.ch_names)
n_samples = raw.n_times
sample_rate = raw.info['sfreq']

# Initialize the EDF file
with pyedflib.EdfWriter(new_file_path, n_channels=n_channels, file_type=pyedflib.FILETYPE_EDFPLUS) as f:
    # Set the header information
    channel_info = []
    data_list = []

    for i, ch_name in enumerate(raw2.ch_names):
        ch_dict = {
            'label': ch_name,
            'dimension': 'uV',
            'sample_frequency': sample_rate,
            'physical_max': raw._data[i].max(),
            'physical_min': raw._data[i].min(),
            'digital_max': 32767,
            'digital_min': -32768,
            'transducer': '',
            'prefilter': ''
        }
        channel_info.append(ch_dict)
        data_list.append(raw._data[i])

    f.setSignalHeaders(channel_info)
    f.writeSamples(data_list)

In [10]:
### this code is or checking digital min digital max

import pyedflib

# Open the EDF file
edf_file = pyedflib.EdfReader(file_path)

# Get the number of channels
n_channels = edf_file.signals_in_file

# Print the digital and physical ranges for each channel
for i in range(n_channels):
    digital_min = edf_file.getDigitalMinimum(i)
    digital_max = edf_file.getDigitalMaximum(i)
    physical_min = edf_file.getPhysicalMinimum(i)
    physical_max = edf_file.getPhysicalMaximum(i)
    channel_label = edf_file.getLabel(i)
    
    print(f"Channel {i} ({channel_label}):")
    print(f"  Digital Min: {digital_min}")
    print(f"  Digital Max: {digital_max}")
    print(f"  Physical Min: {physical_min}")
    print(f"  Physical Max: {physical_max}")
    print()

# Close the EDF file
edf_file.close()

Channel 0 (PSG_F3):
  Digital Min: -32768
  Digital Max: 32767
  Physical Min: -548.417
  Physical Max: 1959.649

Channel 1 (PSG_F4):
  Digital Min: -32768
  Digital Max: 32767
  Physical Min: -1051.04
  Physical Max: 970.8509

Channel 2 (PSG_C3):
  Digital Min: -32768
  Digital Max: 32767
  Physical Min: -3200.0
  Physical Max: 2977.532

Channel 3 (PSG_C4):
  Digital Min: -32768
  Digital Max: 32767
  Physical Min: -3200.0
  Physical Max: 3200.0

Channel 4 (PSG_O1):
  Digital Min: -32768
  Digital Max: 32767
  Physical Min: -340.871
  Physical Max: 359.9176

Channel 5 (PSG_O2):
  Digital Min: -32768
  Digital Max: 32767
  Physical Min: -439.403
  Physical Max: 788.5374

Channel 6 (PSG_EOG):
  Digital Min: -32768
  Digital Max: 32767
  Physical Min: -620.753
  Physical Max: 983.9506

Channel 7 (PSG_EMG):
  Digital Min: -32768
  Digital Max: 32767
  Physical Min: -3200.0
  Physical Max: 3200.0

Channel 8 (PSG_THER):
  Digital Min: -32768
  Digital Max: 32767
  Physical Min: -463.926
  P

### ok, let's do this for all the channels

In [3]:
# Create a new EDF file
def filter_6_channels(sub_no):
    file_path = f'Dataset_clean_for_jupyter/sub-{sub_no}/eeg/sub-{sub_no}_task-Sleep_acq-psg_eeg.edf'
    new_file_path =f'Dataset_clean_for_jupyter/sub-{sub_no}/eeg/sub-{sub_no}_task-Sleep_acq-psg_eeg_6-channels.edf'
    
    raw = mne.io.read_raw_edf(file_path, preload=True)
    raw2= raw.copy().pick(range(6))
    n_samples = raw.n_times
    sample_rate = raw.info['sfreq']
    
    # Initialize the EDF file
    with pyedflib.EdfWriter(new_file_path, n_channels=6, file_type=pyedflib.FILETYPE_EDFPLUS) as f:
        # Set the header information
        channel_info = []
        data_list = []
    
        for i, ch_name in enumerate(raw2.ch_names):
            ch_dict = {
                'label': ch_name,
                'dimension': 'uV',
                'sample_frequency': sample_rate,
                'physical_max': raw._data[i].max(),
                'physical_min': raw._data[i].min(),
                'digital_max': 32767,
                'digital_min': -32768,
                'transducer': '',
                'prefilter': ''
            }
            channel_info.append(ch_dict)
            data_list.append(raw._data[i])
    
        f.setSignalHeaders(channel_info)
        f.writeSamples(data_list)

In [4]:
for i in range(1,129):
    filter_6_channels(i)

Extracting EDF parameters from C:\Users\naikh\SleepResearchCode_experient\Dataset_clean_for_jupyter\sub-1\eeg\sub-1_task-Sleep_acq-psg_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7029503  =      0.000 ... 27458.996 secs...


C:\Users\naikh\anaconda3\envs\PytorchEnvironment\Lib\site-packages\pyedflib\edfwriter.py:134: UserWarning: Physical minimum for channel 0 (PSG_F3) is -0.003199929851056687, which has 21 chars, however, EDF+ can only save 8 chars, will be truncated to -0.00319, some loss of precision is to be expected
  warnings.warn('Physical minimum for channel {} ({}) is {}, which has {} chars, '\
C:\Users\naikh\anaconda3\envs\PytorchEnvironment\Lib\site-packages\pyedflib\edfwriter.py:141: UserWarning: Physical maximum for channel 0 (PSG_F3) is 0.00010226150644693723, which has 22 chars, however, EDF+ can only save 8 chars, will be truncated to 0.000102, some loss of precision is to be expected.
  warnings.warn('Physical maximum for channel {} ({}) is {}, which has {} chars, '\
C:\Users\naikh\anaconda3\envs\PytorchEnvironment\Lib\site-packages\pyedflib\edfwriter.py:134: UserWarning: Physical minimum for channel 1 (PSG_F4) is -0.0031999347552605474, which has 22 chars, however, EDF+ can only save 8 ch

Extracting EDF parameters from C:\Users\naikh\SleepResearchCode_experient\Dataset_clean_for_jupyter\sub-2\eeg\sub-2_task-Sleep_acq-psg_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7014399  =      0.000 ... 27399.996 secs...


C:\Users\naikh\anaconda3\envs\PytorchEnvironment\Lib\site-packages\pyedflib\edfwriter.py:134: UserWarning: Physical minimum for channel 0 (PSG_F3) is -0.00319990750873579, which has 20 chars, however, EDF+ can only save 8 chars, will be truncated to -0.00319, some loss of precision is to be expected
  warnings.warn('Physical minimum for channel {} ({}) is {}, which has {} chars, '\
C:\Users\naikh\anaconda3\envs\PytorchEnvironment\Lib\site-packages\pyedflib\edfwriter.py:141: UserWarning: Physical maximum for channel 0 (PSG_F3) is 0.002540747786984054, which has 20 chars, however, EDF+ can only save 8 chars, will be truncated to 0.002540, some loss of precision is to be expected.
  warnings.warn('Physical maximum for channel {} ({}) is {}, which has {} chars, '\
C:\Users\naikh\anaconda3\envs\PytorchEnvironment\Lib\site-packages\pyedflib\edfwriter.py:134: UserWarning: Physical minimum for channel 1 (PSG_F4) is -0.003199925243411917, which has 21 chars, however, EDF+ can only save 8 chars,

Extracting EDF parameters from C:\Users\naikh\SleepResearchCode_experient\Dataset_clean_for_jupyter\sub-3\eeg\sub-3_task-Sleep_acq-psg_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7659775  =      0.000 ... 29920.996 secs...


...

C:\Users\naikh\anaconda3\envs\PytorchEnvironment\Lib\site-packages\pyedflib\edfwriter.py:134: UserWarning: Physical minimum for channel 0 (PSG_F3) is -0.0005996884234988936, which has 22 chars, however, EDF+ can only save 8 chars, will be truncated to -0.00059, some loss of precision is to be expected
  warnings.warn('Physical minimum for channel {} ({}) is {}, which has {} chars, '\
C:\Users\naikh\anaconda3\envs\PytorchEnvironment\Lib\site-packages\pyedflib\edfwriter.py:141: UserWarning: Physical maximum for channel 0 (PSG_F3) is 0.001069792, which has 11 chars, however, EDF+ can only save 8 chars, will be truncated to 0.001069, some loss of precision is to be expected.
  warnings.warn('Physical maximum for channel {} ({}) is {}, which has {} chars, '\
C:\Users\naikh\anaconda3\envs\PytorchEnvironment\Lib\site-packages\pyedflib\edfwriter.py:134: UserWarning: Physical minimum for channel 1 (PSG_F4) is -0.0005948980462348363, which has 22 chars, however, EDF+ can only save 8 chars, will 

Extracting EDF parameters from C:\Users\naikh\SleepResearchCode_experient\Dataset_clean_for_jupyter\sub-127\eeg\sub-127_task-Sleep_acq-psg_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 6391295  =      0.000 ... 24965.996 secs...


C:\Users\naikh\AppData\Local\Temp\ipykernel_16604\791336486.py:6: RuntimeWarning: Physical range is not defined in following channels:
PSG_PULSE, PSG_BEAT, PSG_SPO2
  raw = mne.io.read_raw_edf(file_path, preload=True)
C:\Users\naikh\AppData\Local\Temp\ipykernel_16604\791336486.py:6: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(file_path, preload=True)
C:\Users\naikh\anaconda3\envs\PytorchEnvironment\Lib\site-packages\pyedflib\edfwriter.py:134: UserWarning: Physical minimum for channel 0 (PSG_F3) is -0.0013850141319905393, which has 22 chars, however, EDF+ can only save 8 chars, will be truncated to -0.00138, some loss of precision is to be expected
  warnings.warn('Physical minimum for channel {} ({}) is {}, which has {} chars, '\
C:\Users\naikh\anaconda3\envs\PytorchEnvironment\Lib\site-packages\pyedflib\edfwriter.py:134: UserWarning: Physical minimum for channel 1 (PSG_F4) is -0.0021886993640039673, wh

Extracting EDF parameters from C:\Users\naikh\SleepResearchCode_experient\Dataset_clean_for_jupyter\sub-128\eeg\sub-128_task-Sleep_acq-psg_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 6309887  =      0.000 ... 24647.996 secs...


C:\Users\naikh\anaconda3\envs\PytorchEnvironment\Lib\site-packages\pyedflib\edfwriter.py:134: UserWarning: Physical minimum for channel 1 (PSG_F4) is -0.0024536756755931944, which has 22 chars, however, EDF+ can only save 8 chars, will be truncated to -0.00245, some loss of precision is to be expected
  warnings.warn('Physical minimum for channel {} ({}) is {}, which has {} chars, '\
C:\Users\naikh\anaconda3\envs\PytorchEnvironment\Lib\site-packages\pyedflib\edfwriter.py:134: UserWarning: Physical minimum for channel 2 (PSG_C3) is -0.002335705596551461, which has 21 chars, however, EDF+ can only save 8 chars, will be truncated to -0.00233, some loss of precision is to be expected
  warnings.warn('Physical minimum for channel {} ({}) is {}, which has {} chars, '\
C:\Users\naikh\anaconda3\envs\PytorchEnvironment\Lib\site-packages\pyedflib\edfwriter.py:134: UserWarning: Physical minimum for channel 3 (PSG_C4) is -0.0024588127740901807, which has 22 chars, however, EDF+ can only save 8 cha